In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"

In [ ]:
# !cp /content/drive/MyDrive/NLP/utils.py ./
!ls

drive  sample_data  utils.py


In [ ]:
#Running on Colab
!pip install keras-nlp
!pip install simpletransformers
!cp /content/drive/MyDrive/NLP/utils.py ./

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successf

In [ ]:
# import py7zr
import time

import keras_nlp
import keras
import tensorflow as tf
from utils import remove_splchars
from pyarrow.parquet import ParquetFile
import multiprocessing
import pyarrow as pa
import numpy as np

from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs
# import tensorflow_datasets as tfds

In [ ]:
BATCH_SIZE = 8
NUM_BATCHES = 600
EPOCHS = 1
MAX_ENCODER_SEQUENCE_LENGTH = 512
MAX_DECODER_SEQUENCE_LENGTH = 128
MAX_GENERATION_LENGTH = 40

In [ ]:
def get_data():
    # data = ParquetFile("./parquet/train.parquet")
    data = ParquetFile("/content/drive/MyDrive/NLP/parquet/train.parquet")
    batch = next(data.iter_batches(batch_size = 1000))
    batch = pa.Table.from_batches([batch]).to_pandas()
    batch.drop(["id"], axis=1, inplace=True)

    #pool = multiprocessing.Pool()
    with multiprocessing.Pool(5) as p:
        batch["article"] = p.map(remove_splchars,batch["article"])


    return batch

def load_ds(data, batch_size):
    values = data.values
    print(values.shape)
    batched = tf.split(values, batch_size, axis=0)


    l = [{"encoder_text": (i[:, 0]), "decoder_text": i[:, 1]} for i in batched]
    return l

In [ ]:
data = get_data()
data.columns = ['input_text', 'target_text']

#Moved load_ds to previous cell for consistency

train_ds = load_ds(data, 125)
# train_ds = train_ds.take(NUM_BATCHES)

(1000, 2)


In [ ]:
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 50
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
    use_cuda=True,
)

model_large = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large-cnn",
    args=model_args,
    use_cuda=True,
)


In [ ]:
print(model_large.predict(
        [
            "1411 EST, 25 October 2013 . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A include fever, tiredness, loss of appetite, nausea and abdominal discomfort. Fargo Catholic Diocese in North Dakota pictured is where the bishop is located"
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


['Bishop John Folda exposed potentially hundreds of church members in Fargo, Grand Forks']


In [ ]:
model.train_model(data, eval_data=data)

ValueError: Output directory (outputs/) already exists and is not empty. Set args.overwrite_output_dir = True to overcome.

In [ ]:
print(model.predict(
        [
            "Tyson is a Cyclops, a son of Poseidon, and Percy Jackson’s half brother. He is the current general of the Cyclopes army."
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:472: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


["Tyson is a Cyclops, a son of Poseidon, and Percy Jackson's"]


In [ ]:
print(model.predict(
        [
            "1411 EST, 25 October 2013 . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A include fever, tiredness, loss of appetite, nausea and abdominal discomfort. Fargo Catholic Diocese in North Dakota pictured is where the bishop is located"
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Bishop John Folda, of North Dakota, is taking time off after being diagnosed']


In [ ]:
import torch
torch.save(model, './model.pth')

# saved_model = torch.load('./model.pth')
saved_model = torch.load('/content/drive/MyDrive/NLP/model_50_epochs.pth')

In [ ]:
import torch

saved_model = torch.load('/content/drive/MyDrive/NLP/model_50_epochs.pth')

print(saved_model.predict(
        [
            "1411 EST, 25 October 2013 . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A include fever, tiredness, loss of appetite, nausea and abdominal discomfort. Fargo Catholic Diocese in North Dakota pictured is where the bishop is located"
        ]\
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:472: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


['Bishop John Folda, of North Dakota, is taking time off after being diagnosed']


In [ ]:

'''
import locale


import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())
'''

'\nimport locale\n\n\nimport locale\ndef getpreferredencoding(do_setlocale = True):\n    return "UTF-8"\nlocale.getpreferredencoding = getpreferredencoding\n\nprint(locale.getpreferredencoding())\n'